In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('df_scaled.csv')
df_pca = pd.read_csv('df_pca6.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23081 entries, 0 to 23080
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          23081 non-null  object 
 1   track_popularity  23081 non-null  float64
 2   danceability      23081 non-null  float64
 3   energy            23081 non-null  float64
 4   loudness          23081 non-null  float64
 5   mode              23081 non-null  int64  
 6   speechiness       23081 non-null  float64
 7   acousticness      23081 non-null  float64
 8   instrumentalness  23081 non-null  float64
 9   liveness          23081 non-null  float64
 10  valence           23081 non-null  float64
 11  tempo             23081 non-null  float64
 12  duration_ms       23081 non-null  float64
 13  year              23081 non-null  float64
 14  genre_edm         23081 non-null  bool   
 15  genre_latin       23081 non-null  bool   
 16  genre_pop         23081 non-null  bool  

In [6]:
df_pca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23081 entries, 0 to 23080
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PC0               23081 non-null  float64
 1   PC1               23081 non-null  float64
 2   PC2               23081 non-null  float64
 3   PC3               23081 non-null  float64
 4   PC4               23081 non-null  float64
 5   PC5               23081 non-null  float64
 6   track_id          17018 non-null  object 
 7   track_popularity  17018 non-null  float64
dtypes: float64(7), object(1)
memory usage: 1.4+ MB
